In [1]:
!nvidia-smi --gpu-reset
!nvidia-smi

Unable to disable persistence mode for GPU 00000000:C2:00.0: Insufficient Permissions
Wed Jan  8 16:31:01 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4090        On  |   00000000:C2:00.0 Off |                  Off |
| 30%   43C    P2             82W /  450W |     587MiB /  24564MiB |      0%      Default |
|                                         |           

In [1]:
import numpy as np
import pandas as pd
from datasets import load_dataset
import os
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModel
import torch
torch.cuda.empty_cache()

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
prefix_path = parent_dir

/home/lielbin/.conda/envs/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.cuda.is_available()

True

In [3]:
df_imdb_sub = pd.read_csv(os.path.join(prefix_path, "data", "imdb_text_subtitles.csv"))

In [4]:
df_imdb_sub.head()

,tconst_episode,tconst_season,seasonNumber,episodeNumber,title_episode,runtimeMinutes,title_series,averageRating_season,numVotes_series,averageRating_episode,numVotes_episode,Year_episode,sub_id,subtitles
0,tt0059753,tt0060028,1,0,The Cage,63.0,Star Trek,8.4,95360.0,7.6,7428.0,1966.0,188032.0,"הכלוב .תבדקו את המעגל- .הכול תקין, אדוני- .אם ..."
1,tt0061027,tt0060028,1,3,Where No Man Has Gone Before,50.0,Star Trek,8.4,95360.0,7.7,6688.0,1966.0,181335.0,",יומן הקפטן .זמן כוכבים 1312.4 ,הבלתי אפשרי קר..."
2,tt0066931,tt1466074,1,3,Dead Weight,76.0,Columbo,8.3,43873.0,6.9,3538.0,1971.0,94598.0,.דוטון .גנרל הוליסטר .היכנס ?רוצה לשתות- .לא. ...
3,tt0073677,tt0071054,3,17,The Secret of Bigfoot: Part 1,47.0,The Six Million Dollar Man,7.1,13511.0,7.6,399.0,1976.0,297110.0,***האיש השווה מיליונים*** תורגם סונכרן וקודד ע...
4,tt0084218,tt0083437,1,1,Knight of the Phoenix: Part 1,45.0,Knight Rider,6.9,38628.0,7.3,2272.0,1982.0,5919.0,"<i>- קונסוליה זרה, ארה""ב -</i> ?תסלחי לי לרגע ..."


In [5]:
example_subtitle = df_imdb_sub['subtitles'].iloc[2]


In [7]:
# example_subtitle

## Summery Hebrew Text with dictalm2.0-instruct Model

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer
# Load model and tokenizer
torch.cuda.is_available()
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "dicta-il/dictalm2.0-instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    torch_dtype=torch.float16,  # Use FP16
    device_map="cuda"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

model.to(device)

Loading checkpoint shards: 100%|██████████| 3/3 [03:34<00:00, 71.34s/it]


MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(33152, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
     

In [10]:
tokenized_output = tokenizer(example_subtitle, max_length=1024, truncation=True, return_tensors="pt")
text_to_summarize = tokenizer.decode(tokenized_output['input_ids'][0], skip_special_tokens=True)
text_to_summarize


'.דוטון .גנרל הוליסטר .היכנס ?רוצה לשתות- .לא. לא תודה, אדוני, אין לי הרבה זמן- ...הייתי מתקשר, אבל- ?אבל מה- .אבל לא רציתי להסתכן בציתות לטלפון ?ציתות .אתה נשמע פראנואידי, קולונל .לא, הגנרל, רק נוקט אמצעי זהירות ?והסיבה לאותה...זהירות הידיעה הגיעה רק הבוקר .הנחיה מיוחדת לכל מפקדי היחידות המפקח הכללי פותח בחקירה רבת היקף .בנושא חוזי אספקת נשק .אני מבין ?אתה בטוח- .כן, אדוני- ,זה חבל מאוד אבל אני משער שזה היה קורה .במוקדם או במאוחר ...אם הם יגלו- .הם לא יגלו- ,המפקד ...אם מישהו יחפור מספיק עמוק ,הם ודאי יגלו שאיפשרתי לחברת הבנייה שלך .לחייב אותנו הון עתק על הוצאות חריגות ...תציע את ההסבר המקובל .קשיים בלתי צפויים בפיתוח מערכות הנשק .זה לא יחזיק מעמד, המפקד ...כאשר החוקר יראה את הצעת המחיר של העיסקה הוא יבין שבאופן מעשי .נתתי לך לגנוב את אותם חוזים .הם יאשימו את שנינו ,לו יהי כדבריך, קולונל .אתה הוא זה שהסכים להצעות המחיר .אתה אישרת את החריגות ,כשכמות כזו גדולה של כסף מעורבת .הם יבינו שהיינו בזה ביחד. אני מסתלק ?או .אני טס הלילה לז\'נבה .ואני מציע לך, גנרל, לעשות כמוני- .יש לי עסק לנהל- 

In [11]:
# Create a prompt for summarization
prompt = f"""
סכם את הטקסט הבא בעברית:
{text_to_summarize}

תקציר:
"""

In [12]:
# Tokenize the prompt
encoded = tokenizer(prompt, return_tensors="pt").to(device)

# Generate the summary
generated_ids = model.generate(
    encoded.input_ids,
    max_new_tokens=250,  # Adjust based on desired summary length
    do_sample=False,  # Deterministic output; set to True for varied results
)

# Decode the generated text
decoded_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

# Extract the summary after "תקציר:"
summary = decoded_text.split("תקציר:")[-1].strip()

# Print the result
print("Generated Summary:")
print(summary)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Generated Summary:
קולונל הוליסטר מגיע לביתו של גנרל דוטון כדי לדון בחוזי אספקת נשק. הגנרל מציע לו לעזוב את המדינה, כיוון שהם עלולים להיחשף. הוליסטר מסרב, ואומר שהוא יכול להיעלם. הגנרל מזהיר אותו שהעולם קטן, והם ימצאו אותו. הוליסטר אומר שהוא יכול לנהוג, והגנרל שואל אותו איך הוא הגיע לשם. הוליסטר אומר שהוא נהג. הגנרל שואל אותו אם הוא ראה מישהו יורה במישהו, והוליסטר אומר שהוא לא ראה כלום. הגנרל אומר שהוא ראה מישהו יורה במישהו, והוליסטר אומר שהוא היה יותר מדי זמן בשמש. הגנרל אומר שהוא צריך לחזור בחזרה, והוליסטר שואל אותו איזו דרך היא בחזרה. הגנרל אומר שהוא לא יודע, והוליסטר אומר שהוא לא יודע. הגנרל אומר שהוא צריך לקחת את


In [13]:
# ---- Get Embeddings ----
# Pass the encoded input through the model to get hidden states
encoded_output = model(
    **encoded,
    output_hidden_states=True,  # Include hidden states
    return_dict=True            # Use dictionary output
)

# Get the last hidden state for the input
input_embeddings = encoded_output.hidden_states[-1]  # Shape: [batch_size, seq_len, hidden_dim]

# To process the generated summary, tokenize it and get embeddings
generated_encoded = tokenizer(summary, return_tensors="pt").to(device)
generated_output = model(
    **generated_encoded,
    output_hidden_states=True,
    return_dict=True
)

# Get the last hidden state for the generated summary
generated_embeddings = generated_output.hidden_states[-1]  # Shape: [batch_size, seq_len, hidden_dim]

# Optionally, get the mean pooling of embeddings as a single vector for the whole input/summary
input_mean_embedding = input_embeddings.mean(dim=1)        # Shape: [batch_size, hidden_dim]
generated_mean_embedding = generated_embeddings.mean(dim=1)  # Shape: [batch_size, hidden_dim]

# Print shapes for verification
print("Input Embeddings Shape:", input_embeddings.shape)
print("Generated Summary Embeddings Shape:", generated_embeddings.shape)
print("Input Mean Embedding Shape:", input_mean_embedding.shape)
print("Generated Mean Embedding Shape:", generated_mean_embedding.shape)

Input Embeddings Shape: torch.Size([1, 1044, 4096])
Generated Summary Embeddings Shape: torch.Size([1, 252, 4096])
Input Mean Embedding Shape: torch.Size([1, 4096])
Generated Mean Embedding Shape: torch.Size([1, 4096])


In [7]:
def summarize_long_text_with_embeddings(text, tokenizer, model, device):
    """
    Summarize a long text by dividing it into chunks, summarizing each chunk,
    and combining the embedding vectors of the summaries.

    Args:
        text (str): The long text to summarize.
        tokenizer: The tokenizer for text processing.
        model: The summarization model.
        device: The device for model inference (e.g., "cpu" or "cuda").

    Returns:
        str: The concatenated summaries of all chunks.
        torch.Tensor: The combined embedding vector for the entire text.
    """
    summaries = []
    summary_embeddings = []

    tokenized_output = tokenizer(text, return_tensors="pt", add_special_tokens=False)
    all_token_ids = tokenized_output['input_ids'][0]
    
    # Process text in chunks of 1024 tokens
    for i in range(0, len(all_token_ids), 1024):
        chunk_token_ids = all_token_ids[i:i + 1024]
        chunk_text = tokenizer.decode(chunk_token_ids, skip_special_tokens=True)
        
        # Summarize the chunk
        prompt = f"""
        סכם את הטקסט הבא בעברית, זה כתוביות של פרק בסדרה, תתקמד בעלילת הפרק ובדמויות:
        {chunk_text}
        
        תקציר:
        """
        
        # Tokenize the prompt
        with torch.no_grad():
            encoded = tokenizer(prompt, return_tensors="pt").to(device)
        
            # Generate the summary
            generated_ids = model.generate(
                encoded.input_ids,
                max_new_tokens=250,
                do_sample=False,
            )
            
            # Decode the generated summary
            decoded_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
            summary = decoded_text.split("תקציר:")[-1].strip()
            summaries.append(summary)
            
            # Get the embedding vector for the summary
            summary_encoded = tokenizer(summary, return_tensors="pt").to(device)
            output = model(**summary_encoded, output_hidden_states=True, return_dict=True)
            summary_embedding = output.hidden_states[-1].mean(dim=1)  # Mean pooling for the summary embedding
            summary_embeddings.append(summary_embedding)

        # Free up GPU memory
        del encoded, generated_ids, summary_encoded, output
        torch.cuda.empty_cache()

    # Concatenate all summaries
    full_summary = " ".join(summaries)
    
    # Combine all summary embeddings using mean pooling
    combined_embedding = torch.mean(torch.stack(summary_embeddings), dim=0)
    
    return full_summary, combined_embedding

In [12]:
full_summary, combined_embedding = summarize_long_text_with_embeddings(example_subtitle, tokenizer, model, device)
print("Final Summary:")
print(full_summary)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad

Final Summary:
[/INST]
הגנרל הוליסטר מגיע לפגישה עם קולונל דוטון, שמסרב להצעה לשתות. הוא מגיע כדי לדון בחשד להונאה בחוזי אספקת נשק, שנחשפה לאחרונה. המפקח הכללי פתח בחקירה מקיפה בנושא, והגנרל חושש שהקולונל מעורב. הקולונל מציע הסבר מקובל, אבל הגנרל מזהיר אותו שהחוקר יבחין בהצעת המחיר המופרזת ויבין שהם היו מעורבים יחד. הקולונל מציע לעזוב את המדינה, אבל הגנרל מזהיר אותו שהעולם קטן והם ימצאו אותו. הקולונל מבטיח להגן על הגנרל, אבל הגנרל לא יכול לקחת את הסיכון.

הקולונל מגיע לפגישה במונית, אבל הוא אומר לגנרל שהוא נהג. הגנרל מבחין באדם שיורה במי [/INST]קולומבו, בלש משטרה, מגיע למעגן סירות כדי לחקור ירי שהתרחש בבית של גנרל הוליסטר. הוא מדבר עם גברת וולטרס, שדיווחה על הירי, ועם בנה, שראה את הירי. קולומבו מקבל את כתובת הבית ואת מספר הטלפון של גברת וולטרס, ושולח מישהו מחקירות רצח כדי לחקור את המקרה. הוא גם מגלה ששני צוערים צעירים נכנסו לבית לפני זמן קצר. [/INST]קולומבו, בלש משטרה, מגיע לבית של גנרל הוליסטר כדי לחקור דיווח על יריות. הוא מגלה שהגנרל מארגן ערב הוקרה לציון 20 שנה לפרישתו, ואוסף מזכרות

In [13]:
# print("Final Summary:")
# print(full_summary)
print("Shape of the tensor:", combined_embedding.shape)


Shape of the tensor: torch.Size([1, 4096])


In [14]:
len(tokenizer.encode(full_summary, add_special_tokens=True))


2921

In [18]:
example_subtitle_2 = df_imdb_sub['subtitles'].iloc[5]


In [19]:
full_summary_2, combined_embedding_2 = summarize_long_text_with_embeddings(example_subtitle_2, tokenizer, model, device)
print("Final Summary 2:")
print(full_summary_2)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attentio

Final Summary 2:
[/INST]
הפרק הראשון של "מסע בין כוכבים: הדור הבא" מתחיל עם הקפטן החדש, ג'ין לוק פיקארד, שמוביל את ספינת החלל אנטרפרייז במשימתה לחקור עולמות חדשים ולגלות תרבויות חדשות. הם מגיעים לכוכב דנב 4, שם הם מגלים בסיס כוכבים שנבנה על ידי תושבי אותו כוכב. בעוד הם מנסים לנהל משא ומתן ידידותי כדי להשתמש בבסיס, הם גם חוקרים את התעלומה של הבסיס.

הצוות עדיין חסר מספר אנשי מפתח, כולל קצין ראשון, אבל הם מצפים להצטרפותו של קומנדר מנוסה בשם ויליאם רייקר. בינתיים, הקפטן פיקארד ממשיך להכיר את הספינה החדשה שלו, USS Enterprise-D, שגדולה ומורכבת יותר ממה שהוא ציפה.

הצוות [/INST]העלילה העיקרית של הפרק הזה היא על כוחו של היישות החייזרית, קיו, ועל הקונפליקט בין בני האדם לבין החייזרים. הקפטן מקבל משימה לחזור לכוכב הבית שלו ולהילחם בקומוניסטים, אבל הוא לא מסכים עם זה. החייזרים טוענים שבני האדם לא השתנו במשך מאות שנים, והם עדיין גזע פראי ומסוכן. הם מזכירים לבני האדם את ההיסטוריה האלימה שלהם, כולל מלחמות על חלוקת משאבים ועל אמונות שבטיות.

הדמויות העיקריות הן הקפטן, ליוטנט וורף, וליוטנט טורס. הקפטן

In [20]:
len(tokenizer.encode(full_summary_2, add_special_tokens=True))

3598

In [12]:
from openpyxl import Workbook, load_workbook

def process_and_save_incrementally(df, tokenizer, model, device, output_path):
    """
    Process a DataFrame to summarize subtitles and save the results incrementally to an Excel file.

    Args:
        df (pd.DataFrame): The input DataFrame with 'sub_id' and 'subtitles' columns.
        tokenizer: The tokenizer for the summarization model.
        model: The summarization model.
        device: The device for model inference (e.g., "cpu" or "cuda").
        output_path (str): The path to save the Excel file.

    Returns:
        None
    """
    # Check if the Excel file exists
    try:
        workbook = load_workbook(output_path)
        sheet = workbook.active
    except FileNotFoundError:
        workbook = Workbook()
        sheet = workbook.active
        # Write headers
        sheet.append(['sub_id', 'full_summary', 'combined_embedding'])

    for index, row in df.iterrows():
        sub_id = row['sub_id']
        subtitles = row['subtitles']

        try:
            # Run the subtitles through the summarization function
            full_summary, combined_embedding = summarize_long_text_with_embeddings(
                text=subtitles,
                tokenizer=tokenizer,
                model=model,
                device=device
            )

            # Convert the embedding tensor to a string for saving
            combined_embedding_str = ','.join(map(str, combined_embedding.cpu().numpy().flatten()))

            # Append results to the Excel sheet
            sheet.append([sub_id, full_summary, combined_embedding_str])
            workbook.save(output_path)  # Save after each record

        except Exception as e:
            # Log the error with the record number and sub_id
            print(f"Error processing record {index + 1} (sub_id: {sub_id}): {e}")
            continue

    print(f"Processing complete. Results saved incrementally to {output_path}")

In [7]:
# output_file = "summarized_subtitles.xlsx"
# process_and_save_incrementally(df_imdb_sub, tokenizer, model, device, output_file)

In [5]:
prefix_path

'/sise/eliorsu-group/lielbin/Courses/The-Art-of-Analyzing-Big-Data'

In [6]:
print(prefix_path + "/data/summarized_subtitles.xlsx")

/sise/eliorsu-group/lielbin/Courses/The-Art-of-Analyzing-Big-Data/data/summarized_subtitles.xlsx


In [7]:
df_summery_embedding = pd.read_excel(prefix_path + "/data/summarized_subtitles.xlsx")
df_summery_embedding.head()

,sub_id,full_summary,combined_embedding
0,188032,"[/INST]הספינה נתקלה באות מצוקה ישן, שנראה שמקו...","-1.188,-1.165,-0.635,1.221,-0.2505,-0.4976,-0...."
1,181335,"[/INST]\nהקפטן של ספינת החלל אנטרפרייז, ג'יימס...","-1.007,-1.57,0.064,1.624,0.01962,0.5225,-0.307..."
2,94598,[/INST]\nהגנרל הוליסטר מגיע לפגישה עם קולונל ד...,"-0.785,-1.588,-0.7036,0.8857,-0.783,-0.18,-0.2..."
3,297110,"[/INST]סטיב אוסטין, אסטרונאוט לשעבר, הוא האדם ...","-0.185,-2.111,-0.05966,1.003,-0.6733,0.2983,-0..."
4,5919,"[/INST]בסיפור, קבוצת אנשים, כולל מייקל, שרה, ...","-1.584,-1.215,-0.5728,1.715,-0.7676,-1.019,1.2..."


In [8]:
len(df_summery_embedding)

34673

In [9]:
len(df_imdb_sub)

67114

In [7]:
df_imdb_sub.loc[11863:11872, 'sub_id']

11863    276556.0
11864    276557.0
11865    228584.0
11866    318989.0
11867    306740.0
11868    298567.0
11869    298566.0
11870    298565.0
11871    298564.0
11872    298591.0
Name: sub_id, dtype: float64

In [6]:
df_summery_embedding.tail(10)

,sub_id,full_summary,combined_embedding
11863,276556,[/INST]\nהפרק מתחיל עם סיכום של אירועים קודמים...,"-1.522,-1.648,-0.693,1.176,-0.8237,-0.4753,0.3..."
11864,276557,[/INST]\nהפרק מתחיל עם סיכום של אירועים קודמים...,"-1.452,-1.851,-0.5615,1.088,-0.8184,-0.1675,-0..."
11865,228584,"[/INST] על פי הטקסט, הסדרה ""מניפסט"" עוקבת אחר ...","-0.94,-2.027,-0.6274,1.291,-0.612,0.2186,-0.78..."
11866,318989,"[/INST]\nהפרק מתמקד בסיפור של אליס, אישה טרנסג...","-2.293,-1.661,-0.2922,0.699,-0.743,-0.04044,0...."
11867,306740,[/INST]\nהפרק מתמקד במשחק בייסבול של קבוצת הנו...,"-1.471,-1.68,-0.5093,1.568,-0.7695,0.6694,-0.4..."
11868,298567,[/INST]\nהפרק הזה מתמקד בקבוצת חברים שמתרגשת מ...,"-0.988,-2.016,-0.9365,1.544,-0.53,0.753,-0.368..."
11869,298566,[/INST]\nהפרק מתמקד בקשיים של היילי וג'ף להיכנ...,"-2.037,-1.413,-0.337,0.9033,-0.535,0.3145,-0.3..."
11870,298565,"[/INST]\nהפרק מתחיל עם מציאת האבן הראשונה, שקר...","-0.546,-1.339,-0.6,0.7314,-0.891,0.4236,-0.58,..."
11871,298564,"[/INST]\nהפרק מתמקד בסיפור של סטן, שמתאמן עם ד...","-1.388,-1.832,-0.2177,1.195,-0.4602,-0.273,-0...."
11872,298591,"[/INST]הפרק מתמקד בפסטיבל ""חוק וסדר"" שמתקיים ב...","-1.066,-2.418,0.12213,1.345,-0.6235,0.2213,-0...."


In [10]:
df_summery_embedding.loc[11863]['full_summary']

'[/INST]\nהפרק מתחיל עם סיכום של אירועים קודמים, כולל הצעת נישואין, הקלטת אלבום, והצעת סיבוב הופעות. הדמויות המרכזיות הן מיק ודיוויד, שמתמודדים עם אתגרים שונים בחייהם האישיים והמקצועיים. מיק מודאגת מהחזרה של הוריה, בעוד שדיוויד מתכונן לקראת פתיחת בית הספר. הם מתמודדים עם בעיות שונות, כמו מציאת מתקן לתבלינים, תכנון ארוחת צהריים, והכנת תצהירים.\n\nהפרק ממשיך עם דיוויד ומיק שמתכוננים לקראת פתיחת בית הספר. הם מודאגים מההורים של מיק שחזרו להיות ביחד, אבל הם מנסים להתמקד בעבודה שלהם. דיוויד מתכונן לקראת פתיחת בית הספר, בעוד שמיק מתכוננת לקראת הצגת המחזה שלה.  נים ובחביתה שום דבר לא באמת מפריע לך, נכון למדת את זה בתור מורה בכיתה ג\' .לא, לא ,האמת היא שבגלל שלמדתי את זה .הפכתי למורה בכיתה ג\' עבדתי על התואר השני שלי ,"בביולוגיה ימית ב"וודס הול ,וביליתי את רוב זמני במכמורתנים ,בניתוח של אוכלוסיות המקרל האטלנטי .ופשוט החלטתי בסוף להחליף קריירה ויתרת על ספירת דגים- .אני יודע שזה מפתיע- לא, חשבתי על חייהם .של האנשים שהיו הכי חשובים לי .חוץ מבני המשפחה שלי, וכולם היו מורים .והבנתי שהחיים לא אמורים 